# LDA Model

## Importing Libraries

In [6]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [7]:
import pandas as pd

data = pd.read_csv('Company_Dataset/dataset_others.csv')
data.head(3)

,Company,job_name,job_link,ML Labeled Function,city,country,Function,employment_type,remote,seniority level,Job Status,Date Reviewed,data analyst,company_Link,job_location,job_details,job_id,posting_error,description
0,Spectrum,"Outside Sales Representative | $5,000 Sign On ...",https://sjobs.brassring.com/TGnewUI/Search/hom...,Sales,Opelika,United States,Full-time,No,Mid-Senior Level,to be reviewed,NaN,NaN,NaN,"$5,000 Sign On Bonus* + $2,500 training pay + ...",d74c82fb-27f8-435c-b54b-14ebccc7e9cd,NaN,https://www.smartrecruiters.com/Humanity/74399...,NaN,NaN
1,Spectrum,Sr Technical Writer,https://sjobs.brassring.com/TGnewUI/Search/hom...,Other,Englewood,United States,Full-time,No,Mid-Senior Level,to be reviewed,NaN,NaN,NaN,JOB SUMMARYThis position is responsible for de...,4175936f-1a2c-4635-a536-4348d5bf89f5,NaN,https://www.smartrecruiters.com/Humanity/74399...,NaN,NaN
2,Spectrum,Advertising Account Executive- New Business,https://sjobs.brassring.com/TGnewUI/Search/hom...,Sales,Bay City,United States,Full-time,No,Mid-Senior Level,to be reviewed,NaN,NaN,NaN,Tenacious go-getter. Inquisitive problem solve...,11a21cd7-86c2-48a4-96b9-83fc59e428c1,NaN,https://www.smartrecruiters.com/Humanity/74399...,NaN,NaN


In [8]:
stopwords = stopwords.words("english")
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
data_desc = data['company_Link']
print(data_desc[0])

$5,000 Sign On Bonus* + $2,500 training pay + unlimited commissionAt A GlanceFull-time territory sales role ideal for sales pros and individuals looking to launch their Sales career who appreciate flexibility, with career advancement opportunities and unlimited commissionA role that is ideal for goal-oriented professionals who thrive on meeting new people, selling our essential services, and being on the moveBenefits include paid comprehensive training, excellent health benefits, paid time off, free Spectrum services, education assistance, and more***Get Up To SpeedThrough virtual classroom training and face-to-face sessions, our hands-on training philosophy partners you with established pros to learn the sales skills needed to close the deals in no time. Throughout your career at Spectrum, you will always have the continuing support and encouragement of your fellow Outside Sales Representative peers, Sales Managers, and company leadership.You Have Unlimited PotentialAs a part of our O

**Lemmatization**

In [11]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

data_desc = data_desc.astype('str')
lemmatized_texts = lemmatization(data_desc)
print (lemmatized_texts[0])

training pay unlimited time territory sale role ideal sale pro individual look launch sale career appreciate flexibility career advancement opportunity unlimited commissiona role be ideal goal orient professional thrive meet new people sell essential service be movebenefit include pay comprehensive training excellent health benefit pay time free service education assistance more***get virtual classroom training face face session hand training philosophy partner establish pro learn sale skill need close deal time career always have continue support encouragement fellow peer sale manager company leadership have unlimited part outside team potential earn be limitless right now we‚Äôre offer training pay unlimited commission!we‚äôll have spectrum training team building be priority want succeed territory sale offer support such peer mentor regular group huddlesongoe cutting edge trainingsolid lead give strong startkeep move motivated dedicated member team have opportunity enhance sale skill

## Tokenization

In [12]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0])

['training', 'pay', 'unlimited', 'time', 'territory', 'sale', 'role', 'ideal', 'sale', 'pro', 'individual', 'look', 'launch', 'sale', 'career', 'appreciate', 'flexibility', 'career', 'advancement', 'opportunity', 'unlimited', 'commissiona', 'role', 'be', 'ideal', 'goal', 'orient', 'professional', 'thrive', 'meet', 'new', 'people', 'sell', 'essential', 'service', 'be', 'movebenefit', 'include', 'pay', 'comprehensive', 'training', 'excellent', 'health', 'benefit', 'pay', 'time', 'free', 'service', 'education', 'assistance', 'more', 'get', 'virtual', 'classroom', 'training', 'face', 'face', 'session', 'hand', 'training', 'philosophy', 'partner', 'establish', 'pro', 'learn', 'sale', 'skill', 'need', 'close', 'deal', 'time', 'career', 'always', 'have', 'continue', 'support', 'encouragement', 'fellow', 'peer', 'sale', 'manager', 'company', 'leadership', 'have', 'unlimited', 'part', 'outside', 'team', 'potential', 'earn', 'be', 'limitless', 'right', 'now', 'we', 'aore', 'offer', 'training', '

## Bigrams and Trigrams

In [13]:
#Bigrams and trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

data_bigrams_trigrams = list(data_bigrams_trigrams)
data_bigrams_trigrams[0]

['training',
 'pay',
 'unlimited',
 'time',
 'territory',
 'sale',
 'role',
 'ideal',
 'sale',
 'pro_individual',
 'look_launch',
 'sale',
 'career_appreciate_flexibility',
 'career',
 'advancement',
 'opportunity',
 'unlimited_commissiona',
 'role',
 'be',
 'ideal',
 'goal',
 'orient',
 'professional',
 'thrive',
 'meet',
 'new',
 'people',
 'sell',
 'essential',
 'service',
 'be',
 'movebenefit',
 'include',
 'pay',
 'comprehensive',
 'training',
 'excellent',
 'health',
 'benefit',
 'pay',
 'time',
 'free',
 'service',
 'education_assistance',
 'more',
 'get_virtual_classroom',
 'training',
 'face',
 'face',
 'session_hand',
 'training',
 'philosophy_partner_establish_pro',
 'learn',
 'sale',
 'skill',
 'need_close_deal',
 'time',
 'career',
 'always',
 'have',
 'continue',
 'support_encouragement_fellow',
 'peer',
 'sale',
 'manager',
 'company',
 'leadership',
 'have',
 'unlimited',
 'part',
 'outside',
 'team',
 'potential',
 'earn',
 'be',
 'limitless_right',
 'now_we_aore',
 'o

## Creating Dictionary

In [14]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

## LDA Topic modeling

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

## Vizualizing the Data

In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

c:\users\arpan\appdata\local\programs\python\python39\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
21    -0.429104  0.015184       1        1  9.534971
28    -0.375994 -0.293333       2        1  8.836045
8     -0.288490 -0.144573       3        1  7.886980
4      0.162178 -0.432719       4        1  5.189727
18    -0.073408 -0.013551       5        1  5.015886
24     0.253298 -0.233708       6        1  4.787289
1      0.248528  0.296011       7        1  4.762142
25     0.097842 -0.091607       8        1  4.167384
26    -0.025252 -0.416690       9        1  4.162707
3     -0.197341 -0.445268      10        1  3.954174
5     -0.273152  0.099955      11        1  3.463053
12    -0.439508  0.149738      12        1  3.302457
11     0.068461  0.190680      13        1  3.056884
7      0.089462 -0.391426      14        1  2.671946
2     -0.031993  0.494600      15        1  2.666382
27    -0.241549  0.253087      16        1  2.638793
23    -0.326125  0.309061      17        1  2.606963
15    -0.410862 -0.191000      18        1  2.417610
0      0.279031  0.148307      19        1  2.234572
6      0.365346 -0.349174      20        1  2.080691
20    -0.131058 -0.141081      21        1  2.063669
14     0.229239  0.455678      22        1  1.992818
9     -0.082435 -0.336145      23        1  1.980506
22     0.132129  0.325886      24        1  1.979357
17    -0.205675  0.448068      25        1  1.615310
16    -0.053913  0.286703      26        1  1.428349
19     0.398021 -0.199740      27        1  1.424294
29     0.468471  0.043205      28        1  1.144450
13     0.331253 -0.032760      29        1  0.653333
10     0.462600  0.196611      30        1  0.281260, topic_info=            Term          Freq         Total Category  logprob  loglift
5             be   7889.000000   7889.000000  Default  30.0000  30.0000
341     customer   7303.000000   7303.000000  Default  29.0000  29.0000
231       client  11544.000000  11544.000000  Default  28.0000  28.0000
241        datum   8078.000000   8078.000000  Default  27.0000  27.0000
229     business   8086.000000   8086.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
0        advance      0.029318    361.484028  Topic30 -11.1913  -3.5461
1       advanced      0.029318    893.116063  Topic30 -11.1913  -4.4506
2    advancement      0.029318    332.184867  Topic30 -11.1913  -3.4616
3         always      0.029318    322.500521  Topic30 -11.1913  -3.4320
4            aos      0.029318   5124.075181  Topic30 -11.1913  -6.1976

[1747 rows x 6 columns], token_table=      Topic      Freq                        Term
term                                             
113       5  0.092391                     ability
113       6  0.203553                     ability
113      28  0.703930                     ability
1332      3  0.983805  ability_identify_roadblock
971       6  0.994088          ability_lift_heavy
...     ...       ...                         ...
323      27  0.412822                         you
424       2  0.430549                    you_aore
424       6  0.287551                    you_aore
424      22  0.073053                    you_aore
424      27  0.206726                    you_aore

[3364 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[22, 29, 9, 5, 19, 25, 2, 26, 27, 4, 6, 13, 12, 8, 3, 28, 24, 16, 1, 7, 21, 15, 10, 23, 18, 17, 20, 30, 14, 11])

In [17]:
#saving graph as html
pyLDAvis.save_html(vis, "lda_final.html")